## 实验：药物研发模拟实验

In [ ]:
# 安装必要的库
!pip install rdkit-pypi matplotlib numpy

In [ ]:
try:
    from rdkit import Chem
    from rdkit.Chem import Draw, Descriptors
    import numpy as np
    import matplotlib.pyplot as plt
except ImportError:
    !pip install rdkit-pypi numpy matplotlib
    from rdkit import Chem
    from rdkit.Chem import Draw, Descriptors
    import numpy as np
    import matplotlib.pyplot as plt

%matplotlib inline

print("=== 交互式药物设计学习演示 ===")
print("欢迎参加药物设计学习！本演示将引导您了解药物发现的基本流程。")
print("您将有机会参与实践环节，亲自体验药物设计的过程。")

In [ ]:
# 第一部分：基础知识介绍
print("\n" + "="*50)
print("第一部分：药物设计基础知识")
print("="*50)

# 展示一些常见药物分子
common_drugs = {
    "阿司匹林 (止痛药)": "CC(=O)Oc1ccccc1C(=O)O",
    "咖啡因 (兴奋剂)": "CN1C=NC2=C1C(=O)N(C(=O)N2C)C",
    "青霉素 (抗生素)": "CC1(C(N2C(S1)C(C2=O)NC(=O)CC3=CC=CC=C3)C(=O)O)C",
    "扑热息痛 (止痛药)": "CC(=O)Nc1ccc(O)cc1"
}

print("\n常见药物分子示例:")
fig, axs = plt.subplots(2, 2, figsize=(10, 8))
axs = axs.ravel()

for i, (name, smiles) in enumerate(common_drugs.items()):
    mol = Chem.MolFromSmiles(smiles)
    img = Draw.MolToImage(mol, size=(300, 300))
    axs[i].imshow(img)
    axs[i].set_title(name, fontsize=10)
    axs[i].axis('off')

plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号
plt.tight_layout()
plt.show()

# 实践环节1：学生尝试输入SMILES
print("\n🔬 实践环节1: 尝试输入SMILES字符串")
print("请尝试输入一个简单的SMILES字符串，例如:")
print("- 水: O")
print("- 乙醇: CCO")
print("- 苯: c1ccccc1")

user_smiles = input("请输入一个SMILES字符串 (直接回车使用默认值CCO): ").strip()
if not user_smiles:
    user_smiles = "CCO"
    print("使用默认值: CCO (乙醇)")

try:
    user_mol = Chem.MolFromSmiles(user_smiles)
    if user_mol:
        print("✓ 成功解析SMILES!")
        display(Draw.MolToImage(user_mol, size=(300, 300)))
        print(f"分子式: {Chem.MolToSmiles(user_mol)}")
    else:
        print("✗ 无法解析SMILES，使用默认值CCO")
        user_mol = Chem.MolFromSmiles("CCO")
        user_smiles = "CCO"
except:
    
    print("✗ 发生错误，使用默认值CCO")
    user_mol = Chem.MolFromSmiles("CCO")
    user_smiles = "CCO"

In [ ]:
# 第二部分：类药性规则学习
print("\n" + "="*50)
print("第二部分：类药性规则 (Lipinski五规则)")
print("="*50)

print("\nLipinski五规则是判断化合物是否可能成为口服药物的经验规则:")
print("1. 分子量 < 500 Da")
print("2. LogP (脂水分配系数) < 5")
print("3. 氢键供体数 < 5")
print("4. 氢键受体数 < 10")
print("5. 可旋转键数 < 10 (有时被忽略)")

# 计算当前分子的性质
mw = Descriptors.MolWt(user_mol)
logp = Descriptors.MolLogP(user_mol)
hbd = Descriptors.NumHDonors(user_mol)
hba = Descriptors.NumHAcceptors(user_mol)
rotatable_bonds = Descriptors.NumRotatableBonds(user_mol)

print(f"\n您输入的分子性质:")
print(f"- 分子量: {mw:.2f} Da {'✓' if mw < 500 else '✗'}")
print(f"- LogP: {logp:.2f} {'✓' if logp < 5 else '✗'}")
print(f"- 氢键供体数: {hbd} {'✓' if hbd < 5 else '✗'}")
print(f"- 氢键受体数: {hba} {'✓' if hba < 10 else '✗'}")
print(f"- 可旋转键数: {rotatable_bonds} {'✓' if rotatable_bonds < 10 else '✗'}")

# 实践环节2：学生预测类药性
print("\n🔬 实践环节2: 预测类药性")
print("基于上述性质，您认为这个分子符合口服药物的类药性规则吗?")
prediction = input("请输入您的预测 (y/n): ").strip().lower()

# 计算实际是否符合规则
pass_rules = sum([mw < 500, logp < 5, hbd < 5, hba < 10]) >= 3

if (prediction == 'y' and pass_rules) or (prediction == 'n' and not pass_rules):
    print("✓ 预测正确!")
else:
    print("✗ 预测不正确!")

print(f"实际结果: {'符合' if pass_rules else '不符合'}类药性规则")

In [ ]:
# 第三部分：分子生成与筛选
print("\n" + "="*50)
print("第三部分：分子生成与筛选")
print("="*50)

# 生成类似分子
def generate_variants(base_mol, n=5):
    """生成基础分子的变体"""
    variants = []
    base_smiles = Chem.MolToSmiles(base_mol)
    
    # 简单的变体生成策略：添加/替换官能团
    modifications = [
        ("C", "CC"), ("C", "O"), ("C", "N"), 
        ("O", "C"), ("N", "C"), ("C", "Cl"), ("C", "F")
    ]
    
    for i in range(n):
        mod_smiles = base_smiles
        # 随机应用1-2个修改
        for _ in range(np.random.randint(1, 3)):
            old, new = modifications[np.random.randint(0, len(modifications))]
            if old in mod_smiles:
                mod_smiles = mod_smiles.replace(old, new, 1)
        
        mol = Chem.MolFromSmiles(mod_smiles)
        if mol:
            variants.append(mol)
    
    return variants

print("\n基于您的分子生成5个变体:")
variants = generate_variants(user_mol, 5)

# 显示变体
img = Draw.MolsToGridImage(variants, molsPerRow=5, subImgSize=(200, 200),
                          legends=[f"变体{i+1}" for i in range(len(variants))])
display(img)

# 实践环节3：学生选择最佳变体
print("\n🔬 实践环节3: 选择最佳变体")
print("请观察上述5个变体，基于类药性规则选择一个您认为最适合作为药物的变体:")

for i, mol in enumerate(variants):
    mw = Descriptors.MolWt(mol)
    logp = Descriptors.MolLogP(mol)
    hbd = Descriptors.NumHDonors(mol)
    hba = Descriptors.NumHAcceptors(mol)
    print(f"{i+1}. 分子量: {mw:.1f}, LogP: {logp:.2f}, HBD: {hbd}, HBA: {hba}")

choice = input("请输入您的选择 (1-5): ").strip()

try:
    choice_idx = int(choice) - 1
    if 0 <= choice_idx < len(variants):
        selected_mol = variants[choice_idx]
        print(f"您选择了变体 {choice}")
        display(Draw.MolToImage(selected_mol, size=(300, 300)))
    else:
        print("无效选择，使用第一个变体")
        selected_mol = variants[0]
except:
    print("输入错误，使用第一个变体")
    selected_mol = variants[0]

In [ ]:
# 第四部分：活性预测与评估
print("\n" + "="*50)
print("第四部分：活性预测与评估")
print("="*50)

# 导入必要的模块
from rdkit.Chem import Descriptors
import matplotlib.pyplot as plt
import numpy as np

# 简化的活性预测函数
def predict_activity(mol):
    """基于分子性质预测生物活性"""
    mw = Descriptors.MolWt(mol)
    logp = Descriptors.MolLogP(mol)
    hbd = Descriptors.NumHDonors(mol)
    hba = Descriptors.NumHAcceptors(mol)
    
    # 简化的活性预测模型
    activity = (0.4 * (1 - min(mw, 500)/500) +  # 分子量越小越好
                0.3 * (1 - abs(logp - 2.5)/3) + # LogP接近2.5最好
                0.2 * (1 - min(hbd, 3)/3) +     # HBD越少越好
                0.1 * (1 - min(hba, 6)/6))      # HBA越少越好
    
    return max(0, min(1, activity))

# 计算所有变体的活性
activities = [predict_activity(mol) for mol in variants]

# 可视化活性比较
plt.figure(figsize=(10, 6))

# 如果没有choice_idx，使用默认值0（第一个变体）
if 'choice_idx' not in locals() or 'choice_idx' not in globals():
    choice_idx = 0

# 确保颜色列表长度与数据匹配
colors = ['lightblue'] * len(activities)
if choice_idx < len(colors):
    colors[choice_idx] = 'orange'

bars = plt.bar(range(1, len(activities)+1), activities, color=colors)
plt.xlabel('变体编号')
plt.ylabel('预测活性')
plt.title('分子变体的预测活性比较')
plt.ylim(0, 1)
plt.axhline(y=0.7, color='red', linestyle='--', label='高活性阈值 (0.7)')
plt.legend()
plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号

# 在柱状图上添加数值标签
for i, v in enumerate(activities):
    plt.text(i+1, v + 0.02, f'{v:.2f}', ha='center')
    
plt.show()

# 找出最佳变体
best_idx = np.argmax(activities)
best_activity = activities[best_idx]
user_activity = activities[choice_idx]

print(f"\n活性预测结果:")
print(f"- 您选择的变体 {choice_idx+1} 的预测活性: {user_activity:.3f}")
print(f"- 最佳变体 {best_idx+1} 的预测活性: {best_activity:.3f}")

if best_idx == choice_idx:
    print("✓ 恭喜! 您选择了活性最高的变体!")
else:
    print("✗ 您选择的不是活性最高的变体。")
    print("最佳变体的结构:")
    display(Draw.MolToImage(variants[best_idx], size=(300, 300)))

In [ ]:
# 第五部分：总结与反思
print("\n" + "="*50)
print("第五部分：总结与反思")
print("="*50)

print("\n🎉 恭喜您完成了药物设计的学习体验!")
print("\n通过本实践，您体验了药物设计的以下关键步骤:")
print("1. 分子表示与SMILES语法")
print("2. 类药性规则评估 (Lipinski五规则)")
print("3. 分子修饰与变体生成")
print("4. 活性预测与分子选择")

print("\n💡 反思问题:")
print("1. 您认为类药性规则在药物设计中的作用是什么?")
print("2. 分子结构的哪些变化可能影响其生物活性?")
print("3. 在真实的药物设计中，还需要考虑哪些因素?")

print("\n📚 扩展学习建议:")
print("- 了解更多药物设计规则: Veber规则, Ghose过滤器等")
print("- 学习分子对接原理与方法")
print("- 探索QSAR (定量构效关系) 模型")

# 保存学生的选择结果
student_results = {
    "初始分子": user_smiles,
    "选择的变体": choice_idx+1 if 'choice_idx' in locals() else 1,
    "预测活性": user_activity,
    "最佳变体": best_idx+1,
    "最佳活性": best_activity
}

print(f"\n您的学习成果:")
for key, value in student_results.items():
    print(f"{key}: {value}")